In [1]:
#https://github.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import rasterio as rio
import requests
# Initialize the Earth Engine module.
ee.Initialize()

In [2]:
#########################################################################
############################ USER INPUTS ################################
#########################################################################
# PATHS
# path to temporary folder to store tif files from gee
TIFpath = 'GEE_Downloads/'

# DOMAIN
# choose the modeling domain
domain = 'WY'

# path to where you want your output met .dat fime
OUT_DEMpath = 'DEM_'+domain+'.asc'
OUT_LCpath = 'NLC2016_'+domain+'.asc'
OUT_LATpath = 'grid_lat.asc'
OUT_LONpath = 'grid_lon.asc'

# # TIME
# # choose if want to set 'manual' or 'auto' date 
# date_flag = 'auto'
# # If you choose 'manual' set your dates below  
# # This will start on the 'begin' date at 0:00 and the last iteration will 
# # be on the day before the 'end' date below.
# st_dt = '2020-04-03'
# ed_dt = '2020-04-04'
#########################################################################

# Download DEM and LC data from GEE

In [3]:
# Download CFSv2 met data function
def get_topoveg(domain, TIFpath):
    
    #create directory for temporary TIFF files
    !mkdir -p $TIFpath

    #path to CSO domains
    domains_resp = requests.get("https://raw.githubusercontent.com/snowmodel-tools/preprocess_python/master/CSO_domains.json")
    domains = domains_resp.json()

    '''
    // These are the min and max corners of your domain in Lat, Long
    // Western Wyoming
    // Input the minimum lat, lower left corner
    '''
    minLat = domains[domain]['Bbox']['latmin']
    #// Input the minimum long, lower left corner
    minLong = domains[domain]['Bbox']['lonmin']
    #// Input the max lat, upper right corner
    maxLat = domains[domain]['Bbox']['latmax']
    #// Input the max Long, upper right corner
    maxLong = domains[domain]['Bbox']['lonmax']


    # This resolution for the NLCD and DEM outputs for the SnowModel domain in meters
    sm_resolution = int(domains[domain]['cellsize'])

    '''// Resolution for the PRISM output. This shoud change by Latitude of the domain
    // because the PRISM product spatial resolution is 2.5 minutes, which equals 150 arc seconds.
    // You can use this arc-second calculator to estimate the correct value for the PRISM resolution by latitude
    // https://opendem.info/arc2meters.html
    // This is one arc-second in meters for 43 degrees N Latitude'''
    one_arcsecond = 22.57
    PRISM_resolution = one_arcsecond * 150

    '''// Define the final output projection using EPSG codes'''
    epsg_code = domains[domain]['mod_proj']

    my_domain = ee.Geometry.Rectangle(**{'coords':[minLong,minLat,maxLong,maxLat],'proj': 'EPSG:4326','geodesic':True,});
    
        #################### adjust for AK ##########################
    #need to specifiy different GEE products for above 60 deg lat
    #if maxLat>60:
    
    # download 30m SRTM data
    #// NOTE: This only covers through 60 degrees latitude. See visualization layers.
    SRTM30 = ee.Image('CGIAR/SRTM90_V4')#ee.Image('USGS/SRTMGL1_003')
    #image = SRTM30.clip(my_domain).unmask()
    filename = os.path.join(TIFpath, 'DEM.tif')
    geemap.ee_export_image(SRTM30, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code)
    
    # download NLCD data
    NLCD = ee.ImageCollection('USGS/NLCD');
    landcover = NLCD.select('landcover');
    #// Define the timeframe of NLCD images to select. Currently, its set to the previous 5 years.
    landcoverfiltered=landcover.filterDate('2015-01-01','2020-01-01');
    #// Create a single image out of the image collection using the most common land cover 
    #// designation from the previous 5 years.
    lcsingle=landcoverfiltered.mode();
    filename = os.path.join(TIFpath, 'NLCD2016.tif')
    geemap.ee_export_image(lcsingle, filename=filename, scale=sm_resolution, region=my_domain, file_per_band=False, crs = epsg_code)

In [4]:
get_topoveg(domain, TIFpath)

Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/DEM.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/GEE_Downloads/NLCD2016.tif


# DEM

In [80]:
# dem
def DEM2SM(INfile, OUTfile):
    src=rio.open(INfile)
    dem=src.read(1)
    #ascii header 
    head = "ncols "+str(np.shape(dem)[1])+"\n" \
    "nrows "+str(np.shape(dem)[0])+"\n" \
    "xllcorner     "+str(int(src.bounds.left))+"\n" \
    "yllcorner     "+str(int(src.bounds.bottom))+"\n" \
    "cellsize      "+str(int(src.res[0]))+"\n" \
    "NODATA_value  -9999"
    np.savetxt(OUTfile, dem, fmt='%d', header = head,comments='')

In [81]:
INfile = TIFpath+'DEM.tif'
DEM2SM(INfile, OUT_DEMpath)

# Landcover 

### NLCD LC Codes

|Code|NLCD2016 Landclass|Code|NLCD2016 Landclass|
| :-: | :-: | :-: | :-: |
|11 | open water |51 | dwarf shrub|
|12| ice / snow |52 | shrub/scrub|
|21 | developed; open space |71 | grassland/herbaceous|
|22 | developed; low intensity|72 | hedge/herbaceous|
|23 | developed; med intensity|73 | lichens|
|24 | developed; high intensity|74 | moss|
|31 | barren; rock, sand, clay|81 | pasture/hay|
|41 | deciduous forest|82 | cultivated crops|
|42 | evergreen forest|90 | woody wetlands|
|43 | mixed shrub|95 | emergent herbaceous wetlands|


### Snowmodel LC Codes

|Code  |Landcover Class |Code  |Landcover Class |
| --- | --- | --- | --- |
|1     | coniferous forest |13    | subalpine meadow  |      
|2     | deciduous forest |14    | tundra (non-tussock) |      
|3     | mixed forest |15    | tundra (tussock) |           
|4     | scattered short-conifer |16    | prostrate shrub tundra | 
|5     | clearcut conifer |17    | arctic gram. wetland |       
|6     | mesic upland shrub |18    | bare |       
|7     | xeric upland shrub |19    | water/possibly frozen |       
|8     | playa shrubland |20    | permanent snow/glacier |         
|9     | shrub wetland/riparian |21    | residential/urban |   
|10    | erect shrub tundra |22    | tall crops |       
|11    | low shrub tundra |23    | short crops |        
|12    | grassland rangeland  |24    | ocean |    

In [78]:
# landcover data 
def LC2SM(INfile,OUTfile):
    src=rio.open(INfile)
    lc=src.read(1)
    #ascii header 
    head = "ncols "+str(np.shape(lc)[1])+"\n" \
    "nrows "+str(np.shape(lc)[0])+"\n" \
    "xllcorner     "+str(int(src.bounds.left))+"\n" \
    "yllcorner     "+str(int(src.bounds.bottom))+"\n" \
    "cellsize      "+str(int(src.res[0]))+"\n" \
    "NODATA_value  -9999"

    #reassign lc from NLCD to SM classes
    DIR=DIR=np.empty([np.shape(lc)[0],np.shape(lc)[1]])
    DIR[lc == 11 ]=24
    DIR[lc == 12 ]=20
    DIR[lc == 21 ]=21
    DIR[lc == 22 ]=21
    DIR[lc == 23 ]=21
    DIR[lc == 24 ]=21
    DIR[lc == 31 ]=18
    DIR[lc == 41 ]=2
    DIR[lc == 42 ]=1
    DIR[lc == 43 ]=6
    DIR[lc == 51 ]=6
    DIR[lc == 52 ]=6
    DIR[lc == 71 ]=12
    DIR[lc == 72 ]=12
    DIR[lc == 73 ]=12
    DIR[lc == 74 ]=12
    DIR[lc == 81 ]=23
    DIR[lc == 82 ]=22
    DIR[lc == 90 ]=9
    DIR[lc == 95 ]=9
    DIR.astype(int)
    np.savetxt(OUTfile, DIR, fmt='%d', header = head,comments='')

In [79]:
INfile = TIFpath+'NLCD2016.tif'
LC2SM(INfile, OUT_LCpath)

# Lat long grids